In [3]:

#################################################
# Interview_1.py
#題目:給定wiki頁面，找出頁面中所有時間，並且以ISO8601日期格式印出並標出位置
#
# 2016 Chi-Lun Huang
#################################################
import requests
from bs4  import BeautifulSoup as bs
import jieba
from operator import itemgetter 
import dateutil.parser as parser
import re
import datetime

# Funtion for ISO8601 and output
def changeFormat(timeString,locationFirst,locationEnd,paragraphy) :

    text = timeString
    date = (parser.parse(text))
    print('Date:',date.isoformat(),' ，Start:',locationFirst+1,'End:',locationEnd+1,' ，Original artical:',paragraphy,'\n')
    
# Funtion for processing chinese date    
def findDate(words,classify) :
    # Block1 (Basic information and Article)
    if classify == '1' :
        for j in range(0,len(words)):
            # Find the words(年/月/日) in sentence
            if len (words) > 0 and ('年' in words[j].text and '月' in words[j].text and '日' in words[j].text):
                #print(words[j].text)
                paragraph = words[j].text
                for k in range (0,len(paragraph)):
                    # Rebuilding chinese date
                    if paragraph[k] == '年':
                        #print(paragraph[k-4:k])
                        # Month ex: 2, 4, 9 
                        if paragraph[k+2] == '月': 
                            #print(paragraph[k+1:k+2])
                            # Day ex: 3, 4, 7 
                            if paragraph[k+4] == '日': 
                                changeFormat(paragraph[k-4:k]+'/'+paragraph[k+1:k+2]+'/'+paragraph[k+3:k+4],k-4,k+4,paragraph)
                            # Day ex: 03, 04, 07 
                            elif paragraph[k+5] == '日':
                                changeFormat(paragraph[k-4:k]+'/'+paragraph[k+1:k+2]+'/'+paragraph[k+3:k+5],k-4,k+5,paragraph)

                        # Month ex: 02, 04, 09 
                        elif paragraph[k+3] == '月':
                            #print(paragraph[k+1:k+3])
                             # Day ex: 3, 4, 7 
                            if paragraph[k+5] == '日': 
                                changeFormat(paragraph[k-4:k]+'/'+paragraph[k+1:k+3]+'/'+paragraph[k+4:k+5],k-4,k+5,paragraph)
                            #Day ex: 03, 04, 07 
                            elif paragraph[k+6] == '日':
                                changeFormat(paragraph[k-4:k]+'/'+paragraph[k+1:k+3]+'/'+paragraph[k+4:k+6],k-4,k+6,paragraph)
    #Block2 (Refrerence)
    elif classify == '2' :
        # Use regular expression to recognize date
        for o in range(0,len(words)): 
            text = words[o].text

            match = re.search(r'\d{4}-\d{2}-\d{2}',text)
            match2 = re.search(r'\d{4}-\d{1}-\d{2}',text)
            match3 = re.search(r'\d{4}-\d{2}-\d{1}',text)
            match4 = re.search(r'\d{4}-\d{1}-\d{1}',text)

            if match  :
                date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').date() 
                
            elif match2 :
                date2 = datetime.datetime.strptime(match2.group(), '%Y-%m-%d').date() 
                
            elif match3 :
                date3 = datetime.datetime.strptime(match3.group(), '%Y-%m-%d').date() 
                
            elif match4 :
                date4 = datetime.datetime.strptime(match4.group(), '%Y-%m-%d').date() 
                   
            #result 
            print('Date:',date.isoformat(),' ，From: 註釋內文',' ，Original artical:',text,'\n')
        
        
        
# To get web content
res = requests.get("https://zh.wikipedia.org/wiki/%E5%8F%B0%E7%81%A3%E7%A9%8D%E9%AB%94%E9%9B%BB%E8%B7%AF%E8%A3%BD%E9%80%A0%E5%85%AC%E5%8F%B8")
soup = bs(res.text,'html.parser')



# Basic information
for vcard in soup.select('.vcard'):
    td = vcard.select('td')
    findDate(td,'1')    

# Article   
for div in soup.select(".mw-content-ltr"):
    p = div.select('p')
    findDate(p,'1')

# Refrerence
for div in soup.select(".references-small"):
    referenceText = div.select(".reference-text")
    findDate(referenceText,'2')

    
    

                        
               




Date: 1994-09-05T00:00:00  ，Start: 10 End: 18  ，Original artical: 臺證所：2330（1994年9月5日上市）
NYSE：TSM 

Date: 1987-02-21T00:00:00  ，Start: 1 End: 10  ，Original artical: 1987年2月21日 (1987-02-21) 

Date: 2013-11-12T00:00:00  ，Start: 1 End: 11  ，Original artical: 2013年11月12日，張忠謀卸下執行長一職，但仍然擔任董事長，由劉德音及魏哲家為總經理暨共同執行長。2009年6月，台積電成立新事業組織，並由蔡力行擔任其總經理，並於2011年8月1日切割成兩家子公司台積固態照明和台積太陽能，並由蔡力行擔任首屆董事長。由於2014年1月蔡力行轉任中華電信董事長，由資深副總經理暨資訊長左大川兼任，近期規劃上市，太陽能由趙應誠擔任總經理，LED固態照明則由陳家湘掌舵，兩位都曾任台積電中國區總經理。台積電全球員工總數約為43,000人。 

Date: 2011-08-01T00:00:00  ，Start: 85 End: 93  ，Original artical: 2013年11月12日，張忠謀卸下執行長一職，但仍然擔任董事長，由劉德音及魏哲家為總經理暨共同執行長。2009年6月，台積電成立新事業組織，並由蔡力行擔任其總經理，並於2011年8月1日切割成兩家子公司台積固態照明和台積太陽能，並由蔡力行擔任首屆董事長。由於2014年1月蔡力行轉任中華電信董事長，由資深副總經理暨資訊長左大川兼任，近期規劃上市，太陽能由趙應誠擔任總經理，LED固態照明則由陳家湘掌舵，兩位都曾任台積電中國區總經理。台積電全球員工總數約為43,000人。 

Date: 2015-01-09T00:00:00  ，Start: 1 End: 9  ，Original artical: 2015年1月9日台積電召開臨時董事會，通過把持有之子公司台積固態照明公司全數股份以8.25 億元售予晶元光電，換算每股售出價格為1.46 元，交易完成後，晶電將持有台積固態照明94% 股權，台積電則完全退出該公司，往後營運將由晶電及固態照明現有團隊主導，原總經理陳家湘轉任至台